In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms



normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std =[0.229, 0.224, 0.225],
)


trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=normalize)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=normalize)

100%|██████████| 170M/170M [00:03<00:00, 43.7MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

from torchvision.models import resnet18, ResNet18_Weights
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

for param in model.parameters():
    param.requires_grad = False

model.fc= nn.Linear(model.fc.in_features, 10)
model = model.to(device)

for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.fc.parameters(), lr=1e-3, weight_decay=0.01)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}], Loss: {running_loss/len(train_loader.dataset):.4f}, Accuracy: {100.*correct/total:.2f}%")

    # Eval
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Test Accuracy: {100.*correct/total:.2f}%")


Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 123MB/s]


Epoch [1], Loss: 1.5564, Accuracy: 54.27%
Test Accuracy: 71.35%
Epoch [2], Loss: 0.9548, Accuracy: 73.00%
Test Accuracy: 75.24%
Epoch [3], Loss: 0.8001, Accuracy: 75.75%
Test Accuracy: 76.85%
Epoch [4], Loss: 0.7301, Accuracy: 77.04%
Test Accuracy: 77.89%
Epoch [5], Loss: 0.6879, Accuracy: 77.52%
Test Accuracy: 78.35%
Epoch [6], Loss: 0.6639, Accuracy: 78.10%
Test Accuracy: 78.69%
Epoch [7], Loss: 0.6449, Accuracy: 78.73%
Test Accuracy: 78.79%
Epoch [8], Loss: 0.6301, Accuracy: 78.89%
Test Accuracy: 79.16%
Epoch [9], Loss: 0.6188, Accuracy: 79.37%
Test Accuracy: 79.39%
Epoch [10], Loss: 0.6104, Accuracy: 79.45%
Test Accuracy: 79.54%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

model = models.resnet18(num_classes=100)
model.load_state_dict(torch.load('/content/drive/MyDrive/resnet18_checkpoints/resnet18_latest.pth', map_location=device))

for param in model.parameters():
    param.requires_grad = False

model.fc= nn.Linear(model.fc.in_features, 10)
model = model.to(device)

for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.fc.parameters(), lr=1e-3, weight_decay=0.01)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}], Loss: {running_loss/len(train_loader.dataset):.4f}, Accuracy: {100.*correct/total:.2f}%")

    # Eval
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Test Accuracy: {100.*correct/total:.2f}%")


Using device: cuda


100%|██████████| 170M/170M [00:04<00:00, 40.4MB/s]


Epoch [1], Loss: 1.3438, Accuracy: 56.56%
Test Accuracy: 66.37%
Epoch [2], Loss: 0.9578, Accuracy: 67.37%
Test Accuracy: 68.90%
Epoch [3], Loss: 0.8889, Accuracy: 68.87%
Test Accuracy: 69.70%
Epoch [4], Loss: 0.8564, Accuracy: 69.78%
Test Accuracy: 70.28%
Epoch [5], Loss: 0.8345, Accuracy: 70.52%
Test Accuracy: 70.86%
Epoch [6], Loss: 0.8212, Accuracy: 70.86%
Test Accuracy: 71.38%
Epoch [7], Loss: 0.8116, Accuracy: 71.40%
Test Accuracy: 71.49%
Epoch [8], Loss: 0.8005, Accuracy: 71.58%
Test Accuracy: 72.05%
Epoch [9], Loss: 0.7971, Accuracy: 71.75%
Test Accuracy: 72.29%
Epoch [10], Loss: 0.7863, Accuracy: 72.23%
Test Accuracy: 72.36%
